In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)

c:\Users\roseh\Desktop\study\deep_learning\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\roseh\Desktop\study\deep_learning\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\roseh\Desktop\study\deep_learning\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\roseh\Desktop\study\deep_lea

In [6]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X=tf.placeholder("float32", [None,4])
Y=tf.placeholder("float32", [None,3])
n_classes = 3

W=tf.Variable(tf.random_normal([4,n_classes]), name='weight')
b=tf.Variable(tf.random_normal([n_classes]), name='bias')

hypothesis=tf.nn.softmax(tf.matmul(X,W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step%200==0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('--------------')
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))

0 3.838109
200 0.551395
400 0.45051903
600 0.37469992
800 0.3032415
1000 0.23811388
1200 0.214684
1400 0.19588983
1600 0.18003407
1800 0.1664806
2000 0.15476777
--------------
[[3.3856363e-03 9.9660301e-01 1.1331195e-05]] [1]
--------------
[[0.88455176 0.09963025 0.01581793]] [0]
--------------
[[1.5725437e-08 3.6159108e-04 9.9963844e-01]] [2]
--------------
[[3.3856363e-03 9.9660301e-01 1.1331195e-05]
 [8.8455176e-01 9.9630252e-02 1.5817925e-02]
 [1.5725437e-08 3.6159108e-04 9.9963844e-01]] [1 0 2]


In [8]:
xy = np.loadtxt('data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes) #label convert to one_hot
print("one_hot:", Y_one_hot) #input [[1],[0]] -> output [[[0,1]],[[1,0]]]
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) #we want [[0,1],[1,0]], -1 means None, reshape(data, [shape])
print("reshape one_hot:", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

#cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot) #it is more comfortable
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1) #ouput(y_bar) to one hot idx
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #cmp idx
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    pred = sess.run(prediction, feed_dict={X: x_data})

    for p, y in zip(pred, y_data.flatten()): #[[0],[1]] -> [0,1]
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Step:     0	Cost: 6.321	Acc: 24.75%
Step:   100	Cost: 0.696	Acc: 83.17%
Step:   200	Cost: 0.466	Acc: 84.16%
Step:   300	Cost: 0.344	Acc: 86.14%
Step:   400	Cost: 0.265	Acc: 92.08%
Step:   500	Cost: 0.211	Acc: 95.05%
Step:   600	Cost: 0.174	Acc: 97.03%
Step:   700	Cost: 0.147	Acc: 97.03%
Step:   800	Cost: 0.127	Acc: 98.02%
Step:   900	Cost: 0.112	Acc: 98.02%
Step:  1000	Cost: 0.101	Acc: 99.01%
Step:  1100	Cost: 0.091	Acc: 100.00%
Step:  1200	Cost: 0.083	Acc: 100.00%
Step:  1300	Cost: 0.077	Acc: 100.00%
Step:  1400	Cost: 0.071	Acc: 100.00%
Step:  1500	Cost: 0.067	Acc: 100.00%
Step:  1600	Cost: 0.063	Acc: 100.00%
Step:  1700	Cost: 0.059	Acc: 100.00%
Step:  1800	Cost: 0.056	Acc: 100.00%
Step:  1900	Cost: 0.053	Acc: 100.00%
Step:  2000	Cost: 0.050	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction